In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [ ]:
with open('../data/teis_final_data.pickle', 'rb') as file:
    teis_merge2 = pickle.load(file)

In [ ]:
teis_merge2

### Bullet 5
TEIS provides services to eligible infants and toddlers from birth to age 3 (for this question, only consider cases which have reached the IFSP phase).

    -Are there any POE or service coordinators with high rates of early closure (a child exiting before reaching age 3)? 


In [ ]:
ifsp = teis_merge2.loc[teis_merge2['Child Phase'] == 'IFSP']
ifsp.info()

In [ ]:
ifsp_exit = ifsp.dropna(subset=['Exit Date'])
ifsp_exit.info()

In [ ]:
ifsp_exit = ifsp_exit.loc[ifsp_exit['Referral Age'] > '0 days']
ifsp_exit.info()

In [ ]:
all_early_exits = ifsp_exit.loc[ifsp_exit['Exit Age'] < '1092 days']
all_early_exits

In [ ]:
early_exits = all_early_exits.loc[~all_early_exits['Exit Reason'].isin(['618 - Completion of IFSP/ no longer requires services',
                                                                      '618 - Moved out of state', '618 - Deceased'])]
early_exits

In [ ]:
POE_exits = ifsp_exit.groupby('POE')['Child ID'].count()
POE_exits = POE_exits.to_frame().reset_index()
POE_exits = POE_exits.rename(columns = {'Child ID' : 'Total Exits'})
POE_exits

In [ ]:
POE_early_exits = early_exits.groupby('POE')['Child ID'].count()
POE_early_exits = POE_early_exits.to_frame().reset_index()
POE_early_exits = POE_early_exits.rename(columns = {'Child ID' : 'Total Early Exits'})
POE_early_exits

In [ ]:
POE_early_exit_rate = pd.merge(POE_exits, POE_early_exits)
POE_early_exit_rate['Early Exit Rate'] = round((POE_early_exit_rate['Total Early Exits']/
                                                POE_early_exit_rate['Total Exits'])*100, 2)
POE_early_exit_rate = POE_early_exit_rate.sort_values('Early Exit Rate')
POE_early_exit_rate

In [ ]:
POE_early_exit_rate['Total Early Exits'].sum()/POE_early_exit_rate['Total Exits'].sum()

In [ ]:
full_palette = ["#D9ED92", "#B5E48C", "#99D98C", "#76C893", "#52B69A", "#34A0A4", "#168AAD", "#1A759F", "#1E6091", "#184E77"]

main_palette = ["#76C893", "#1E6091"]

In [ ]:
sns.set_theme(style="whitegrid")
eepoe = sns.barplot(x="Early Exit Rate", y="POE", data = POE_early_exit_rate, palette=full_palette)
plt.xlabel("Early Exit Rate")
plt.ylabel("Point of Entry Office")
eepoe.figure.set_size_inches(10, 8)
eepoe.axvline(37.65, linestyle = '--', color='#184E77');
#plt.savefig('../data/eepoe.png', transparent=True)

In [ ]:
POE_svc_early_exits = early_exits.groupby(['POE', 'Service Coordinator'])['Child ID'].count()
POE_svc_early_exits = POE_svc_early_exits.to_frame().reset_index()
POE_svc_early_exits = POE_svc_early_exits.rename(columns = {'Child ID' : 'Total Early Exits'})
POE_svc_early_exits

In [ ]:
POE_svc_exits = ifsp_exit.groupby(['POE', 'Service Coordinator'])['Child ID'].count()
POE_svc_exits = POE_svc_exits.to_frame().reset_index()
POE_svc_exits = POE_svc_exits.rename(columns = {'Child ID' : 'Total Exits'})
POE_svc_exits = POE_svc_exits.loc[POE_svc_exits['Total Exits'] > 3]
POE_svc_exits

In [ ]:
POE_svc_exits['Service Coordinator'].value_counts()

In [ ]:
POE_svc_early_exit_rate = pd.merge(POE_svc_exits, POE_svc_early_exits,  how='left')
POE_svc_early_exit_rate['Early Exit Rate'] = round((POE_svc_early_exit_rate['Total Early Exits']/
                                                POE_svc_early_exit_rate['Total Exits'])*100, 2)
POE_svc_early_exit_rate = POE_svc_early_exit_rate.fillna(0)
POE_svc_early_exit_rate.sort_values('Early Exit Rate', ascending=False)

In [ ]:
UC_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'UC'].nlargest(5, 'Early Exit Rate')
FT_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'FT'].nlargest(5, 'Early Exit Rate')
NW_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'NW'].nlargest(5, 'Early Exit Rate')
ET_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'ET'].nlargest(5, 'Early Exit Rate')
GN_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'GN'].nlargest(5, 'Early Exit Rate')
SW_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'SW'].nlargest(5, 'Early Exit Rate')
SE_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'SE'].nlargest(5, 'Early Exit Rate')
SC_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'SC'].nlargest(5, 'Early Exit Rate')
MD_early_exits = POE_svc_early_exit_rate.loc[POE_svc_early_exit_rate['POE'] == 'MD'].nlargest(5, 'Early Exit Rate')
POE_svc_high_ee = pd.concat([UC_early_exits, FT_early_exits, NW_early_exits, ET_early_exits, GN_early_exits,
                              SW_early_exits, SE_early_exits, SC_early_exits, MD_early_exits])
POE_svc_high_ee

In [ ]:
sns.set_theme(style="whitegrid")
heeft = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = FT_early_exits, color='#52B69A')
heeft.axvline(38.25, linestyle = '--', color='#184E77')
plt.title('First Tennessee - Overall Rate 38.25%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heeft.set(xlim=(0, 100))
heeft.figure.set_size_inches(6, 4);
#plt.savefig('../data/heeft.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heenw = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = NW_early_exits, color='#1A759F')
heenw.axvline(44.86, linestyle = '--', color='#184E77')
plt.title('Northwest - Overall Rate 44.86%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heenw.set(xlim=(0, 100))
heenw.figure.set_size_inches(6, 4);
#plt.savefig('../data/heenw.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heeet = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = ET_early_exits, color='#168AAD')
heeet.axvline(40.35, linestyle = '--', color='#184E77')
plt.title('East Tennessee - Overall Rate 40.35%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heeet.set(xlim=(0, 100))
heeet.figure.set_size_inches(6, 4);
#plt.savefig('../data/heeet.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heegn = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = GN_early_exits, color='#D9ED92')
heegn.axvline(31.39, linestyle = '--', color='#184E77')
plt.title('Greater Nashville - Overall Rate 31.39%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heegn.set(xlim=(0, 100))
heegn.figure.set_size_inches(6, 4);
#plt.savefig('../data/heegn.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heemd = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = MD_early_exits, color='#76C893')
heemd.axvline(36.83, linestyle = '--', color='#184E77')
plt.title('Memphis Delta - Overall Rate 36.83%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heemd.set(xlim=(0, 100))
heemd.figure.set_size_inches(6, 4);
#plt.savefig('../data/heemd.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heese = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = SE_early_exits, color='#99D98C')
heese.axvline(36.33, linestyle = '--', color='#184E77')
plt.title('Southeast - Overall Rate 36.33%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heese.set(xlim=(0, 100))
heese.figure.set_size_inches(6, 4);
#plt.savefig('../data/heese.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heesc = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = SC_early_exits, color='#B5E48C')
heesc.axvline(33.75, linestyle = '--', color='#184E77')
plt.title('South Central - Overall Rate 33.75%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heesc.set(xlim=(0, 100))
heesc.figure.set_size_inches(6, 4);
#plt.savefig('../data/heesc.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heesw = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = SW_early_exits, color='#1E6091')
heesw.axvline(51.8, linestyle = '--', color='#184E77')
plt.title('Southwest - Overall Rate 51.8%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heesw.set(xlim=(0, 100))
heesw.figure.set_size_inches(6, 4);
#plt.savefig('../data/heesw.png', bbox_inches = 'tight', transparent=True)

In [ ]:
sns.set_theme(style="whitegrid")
heeuc = sns.barplot(x="Early Exit Rate", y="Service Coordinator", data = UC_early_exits, color='#34A0A4')
heeuc.axvline(39.44, linestyle = '--', color='#184E77')
plt.title('Upper Cumberland - Overall Rate 39.44%', fontsize=14)
plt.xlabel("Early Exit Rate")
plt.ylabel("Service Coordinator")
heeuc.set(xlim=(0, 100))
heeuc.figure.set_size_inches(6, 4);
#plt.savefig('../data/heeuc.png', bbox_inches = 'tight', transparent=True)

In [ ]:
UC_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'UC') & (POE_svc_early_exit_rate['Early Exit Rate'] > 39.44))]
FT_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'FT') & (POE_svc_early_exit_rate['Early Exit Rate'] > 38.25))]
NW_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'NW') & (POE_svc_early_exit_rate['Early Exit Rate'] > 44.86))]
ET_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'ET') & (POE_svc_early_exit_rate['Early Exit Rate'] > 40.35))]
GN_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'GN') & (POE_svc_early_exit_rate['Early Exit Rate'] > 31.39))]
SW_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'SW') & (POE_svc_early_exit_rate['Early Exit Rate'] > 51.8))]
SE_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'SE') & (POE_svc_early_exit_rate['Early Exit Rate'] > 36.33))]
SC_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'SC') & (POE_svc_early_exit_rate['Early Exit Rate'] > 33.75))]
MD_aa_early_exits = POE_svc_early_exit_rate.loc[((POE_svc_early_exit_rate['POE'] == 'MD') & (POE_svc_early_exit_rate['Early Exit Rate'] > 36.83))]
POE_svc_abavg = pd.concat([UC_aa_early_exits, FT_aa_early_exits, NW_aa_early_exits, ET_aa_early_exits, 
                           GN_aa_early_exits, SW_aa_early_exits, SE_aa_early_exits, SC_aa_early_exits, MD_aa_early_exits])
#POE_svc_abavg

In [ ]:
POE_abavg_count = POE_svc_abavg.groupby('POE')['Service Coordinator'].count()
POE_abavg_count = POE_abavg_count.to_frame().reset_index()
POE_abavg_count = POE_abavg_count.rename(columns={'Service Coordinator':'Total Service Coordinators w/ high EER'})
POE_abavg_count

In [ ]:
POE_svc_abavg.groupby('POE').describe()

In [ ]:
POE_svc_exits.groupby('POE').describe()

In [ ]:
POE_svc_count = POE_svc_exits.groupby('POE')['Service Coordinator'].count()
POE_svc_count = POE_svc_count.to_frame().reset_index()
POE_svc_count = POE_svc_count.rename(columns={'Service Coordinator':'Total Service Coordinators'})
POE_svc_count

In [ ]:
POE_aasvc_rate = pd.merge(POE_svc_count, POE_abavg_count)
POE_aasvc_rate['Percent of Service Coordinators with high EE rate'] = round((POE_aasvc_rate['Total Service Coordinators w/ high EER']/
                                                POE_aasvc_rate['Total Service Coordinators'])*100, 2)
POE_aasvc_rate = POE_aasvc_rate.sort_values('Percent of Service Coordinators with high EE rate')
POE_aasvc_rate

In [ ]:
POE_aasvc_rate['Total Service Coordinators w/ high EER'].sum() / POE_aasvc_rate['Total Service Coordinators'].sum()